In [8]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [9]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "create me a backend for springboot to manage my notes"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user',
   'content': 'create me a backend for springboot to manage my notes'}]}

In [10]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-D5Z8BHbpDfPj2uDnKDg3WuTtY9UNx',
 'object': 'chat.completion',
 'created': 1770219055,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Here’s a ready-to-run Spring Boot REST backend to manage notes. It includes CRUD (Create, Read, Update, Delete), a simple search, and dev-friendly H2 in-memory database with JPA auditing (createdAt, updatedAt).\n\nProject structure (Maven)\n- pom.xml\n- src/main/java/... (several packages)\n- src/main/resources/application.properties\n\nFiles (code included)\n\n1) pom.xml\n- A minimal Spring Boot setup with Web, JPA, H2, and Lombok (reduces boilerplate)\n\n<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"\n  xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">\n  <modelVersion>4.0.0</modelVersion>\n\n  <groupId>com.example</groupId>\n  <artifactId>notes-api</artifactId>\n 

In [11]:
summary = response.json()["choices"][0]["message"]["content"]

In [12]:
# Step 4: Print/display the result
def display_summary():
    """Get and display the summary in Markdown format."""
    
    try:
        display(Markdown(summary))  # For Jupyter notebooks
    except:
        print(summary)  # For regular Python scripts
    
    return summary

    

In [13]:
summary = display_summary()

Here’s a ready-to-run Spring Boot REST backend to manage notes. It includes CRUD (Create, Read, Update, Delete), a simple search, and dev-friendly H2 in-memory database with JPA auditing (createdAt, updatedAt).

Project structure (Maven)
- pom.xml
- src/main/java/... (several packages)
- src/main/resources/application.properties

Files (code included)

1) pom.xml
- A minimal Spring Boot setup with Web, JPA, H2, and Lombok (reduces boilerplate)

<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>

  <groupId>com.example</groupId>
  <artifactId>notes-api</artifactId>
  <version>0.0.1-SNAPSHOT</version>
  <name>notes-api</name>

  <parent>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-parent</artifactId>
    <version>3.1.2</version>
    <relativePath/> 
  </parent>

  <properties>
    <java.version>17</java.version>
  </properties>

  <dependencies>
    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-web</artifactId>
    </dependency>

    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-data-jpa</artifactId>
    </dependency>

    <dependency>
      <groupId>com.h2database</groupId>
      <artifactId>h2</artifactId>
      <scope>runtime</scope>
    </dependency>

    <dependency>
      <groupId>org.projectlombok</groupId>
      <artifactId>lombok</artifactId>
      <optional>true</optional>
    </dependency>
  </dependencies>

  <build>
    <plugins>
      <plugin>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-maven-plugin</artifactId>
      </plugin>
    </plugins>
  </build>
</project>

2) application.properties
- H2 in-memory DB and JPA auditing enabled

spring.datasource.url=jdbc:h2:mem:notesdb;DB_CLOSE_DELAY=-1;DB_CLOSE_ON_EXIT=FALSE
spring.datasource.driver-class-name=org.h2.Driver
spring.datasource.username=sa
spring.datasource.password=
spring.jpa.database-platform=org.hibernate.dialect.H2Dialect
spring.jpa.hibernate.ddl-auto=update
spring.h2.console.enabled=true
spring.jpa.show-sql=true
spring.jpa.properties.hibernate.dialect=org.hibernate.dialect.H2Dialect

3) NotesApplication (main)
package com.example.notes;

import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;
import org.springframework.data.jpa.repository.config.EnableJpaAuditing;

@SpringBootApplication
@EnableJpaAuditing
public class NotesApplication {
  public static void main(String[] args) {
    SpringApplication.run(NotesApplication.class, args);
  }
}

4) Note entity (with auditing)
package com.example.notes.model;

import javax.persistence.*;
import javax.validation.constraints.NotBlank;
import java.time.LocalDateTime;

import lombok.Data;
import lombok.NoArgsConstructor;
import lombok.AllArgsConstructor;

import org.springframework.data.annotation.CreatedDate;
import org.springframework.data.annotation.LastModifiedDate;
import org.springframework.data.jpa.domain.support.AuditingEntityListener;

@Data
@Entity
@Table(name = "notes")
@EntityListeners(AuditingEntityListener.class)
@NoArgsConstructor
@AllArgsConstructor
public class Note {
  @Id
  @GeneratedValue(strategy = GenerationType.IDENTITY)
  private Long id;

  @NotBlank
  private String title;

  @Lob
  private String content;

  @CreatedDate
  private LocalDateTime createdAt;

  @LastModifiedDate
  private LocalDateTime updatedAt;
}

5) NoteRepository
package com.example.notes.repository;

import org.springframework.data.jpa.repository.JpaRepository;
import com.example.notes.model.Note;
import java.util.List;

public interface NoteRepository extends JpaRepository<Note, Long> {
  List<Note> findByTitleContainingIgnoreCaseOrContentContainingIgnoreCase(String title, String content);
}

6) NoteRequest DTO
package com.example.notes.dto;

import javax.validation.constraints.NotBlank;
import lombok.Data;

@Data
public class NoteRequest {
  @NotBlank
  private String title;

  private String content;
}

7) NoteResponse DTO
package com.example.notes.dto;

import java.time.LocalDateTime;
import lombok.Data;

@Data
public class NoteResponse {
  private Long id;
  private String title;
  private String content;
  private LocalDateTime createdAt;
  private LocalDateTime updatedAt;

  public NoteResponse(com.example.notes.model.Note note) {
    this.id = note.getId();
    this.title = note.getTitle();
    this.content = note.getContent();
    this.createdAt = note.getCreatedAt();
    this.updatedAt = note.getUpdatedAt();
  }
}

8) NoteService
package com.example.notes.service;

import org.springframework.stereotype.Service;
import com.example.notes.repository.NoteRepository;
import com.example.notes.model.Note;
import com.example.notes.dto.NoteRequest;
import java.util.List;
import java.util.Optional;
import org.springframework.transaction.annotation.Transactional;
import org.springframework.data.domain.Sort;
import org.springframework.web.server.ResponseStatusException;
import org.springframework.http.HttpStatus;

@Service
public class NoteService {
  private final NoteRepository repository;

  public NoteService(NoteRepository repository) {
    this.repository = repository;
  }

  public List<Note> getAll() {
    return repository.findAll(Sort.by(Sort.Direction.DESC, "updatedAt"));
  }

  public Optional<Note> getById(Long id) {
    return repository.findById(id);
  }

  @Transactional
  public Note create(NoteRequest request) {
    Note note = new Note();
    note.setTitle(request.getTitle());
    note.setContent(request.getContent());
    return repository.save(note);
  }

  @Transactional
  public Note update(Long id, NoteRequest request) {
    Note existing = repository.findById(id).orElseThrow(
      () -> new ResponseStatusException(HttpStatus.NOT_FOUND, "Note not found")
    );
    existing.setTitle(request.getTitle());
    existing.setContent(request.getContent());
    return repository.save(existing);
  }

  @Transactional
  public void delete(Long id) {
    if (!repository.existsById(id)) {
      throw new ResponseStatusException(HttpStatus.NOT_FOUND, "Note not found");
    }
    repository.deleteById(id);
  }

  public List<Note> search(String query) {
    return repository.findByTitleContainingIgnoreCaseOrContentContainingIgnoreCase(query, query);
  }
}

9) NoteController
package com.example.notes.controller;

import org.springframework.web.bind.annotation.*;
import org.springframework.http.ResponseEntity;
import org.springframework.http.HttpStatus;
import org.springframework.web.server.ResponseStatusException;

import javax.validation.Valid;

import java.util.List;
import java.util.stream.Collectors;

import com.example.notes.service.NoteService;
import com.example.notes.dto.NoteRequest;
import com.example.notes.dto.NoteResponse;
import com.example.notes.model.Note;

@RestController
@RequestMapping("/notes")
public class NoteController {
  private final NoteService service;

  public NoteController(NoteService service) {
    this.service = service;
  }

  @PostMapping
  public ResponseEntity<NoteResponse> create(@Valid @RequestBody NoteRequest request) {
    Note note = service.create(request);
    return new ResponseEntity<>(new NoteResponse(note), HttpStatus.CREATED);
  }

  @GetMapping
  public List<NoteResponse> getAll() {
    return service.getAll().stream()
      .map(NoteResponse::new)
      .collect(Collectors.toList());
  }

  @GetMapping("/{id}")
  public ResponseEntity<NoteResponse> get(@PathVariable Long id) {
    Note note = service.getById(id).orElseThrow(
      () -> new ResponseStatusException(HttpStatus.NOT_FOUND, "Note not found")
    );
    return ResponseEntity.ok(new NoteResponse(note));
  }

  @PutMapping("/{id}")
  public ResponseEntity<NoteResponse> update(@PathVariable Long id, @Valid @RequestBody NoteRequest request) {
    Note updated = service.update(id, request);
    return ResponseEntity.ok(new NoteResponse(updated));
  }

  @DeleteMapping("/{id}")
  public ResponseEntity<Void> delete(@PathVariable Long id) {
    service.delete(id);
    return ResponseEntity.noContent().build();
  }

  @GetMapping("/search")
  public List<NoteResponse> search(@RequestParam("q") String q) {
    return service.search(q).stream()
      .map(NoteResponse::new)
      .collect(Collectors.toList());
  }
}

How to run
- In project root, run:
  - mvn clean package
  - java -jar target/notes-api-0.0.1-SNAPSHOT.jar
  Or use the Spring Boot DevTools with your IDE to run directly.

- Endpoints (examples)
  - POST /notes
    Body: { "title": "My first note", "content": "Hello world" }
  - GET /notes
  - GET /notes/{id}
  - PUT /notes/{id}
    Body: { "title": "Updated", "content": "New content" }
  - DELETE /notes/{id}
  - GET /notes/search?q=hello

Tips and enhancements (optional)
- Swap to a real DB:
  - Update application.properties with a real DB (PostgreSQL, MySQL, etc.)
- Authentication:
  - Add Spring Security if you want per-user notes.
- Frontend:
  - Consume the REST API from a frontend (React, Vue, etc.).
- Validation:
  - Add more constraints on title/content as needed.
- Pagination:
  - Add pageable results for GET /notes if you accumulate many notes.
- Docker:
  - Dockerize the app and connect to a DB container if needed.

If you’d like, I can tailor this to a specific DB, add user accounts (per-note ownership), or include unit/integration tests.

In [14]:
# The shortest way to get the summary

# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Fun fact: Bananas are technically berries, but strawberries aren’t. Botanically, a berry is a fruit that develops from a single flower with one ovary and contains seeds inside the fleshy body. Bananas fit that bill, while strawberries are actually aggregates of multiple tiny fruits (the seeds on the outside are separate fruits, and the fleshy part we eat is an enlarged receptacle). Want another fun fact?'

In [18]:
## And Ollama also gives an OpenAI compatible endpoint

In [ ]:
requests.get("http://localhost:11434").content

b'Ollama is running'

In [19]:
!ollama pull llama3.2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 622 KB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 2.2 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 4.8 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 8.6 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 9.1 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  11 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  15 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  17 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕       

In [21]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL,api_key='ollama')
response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Here's one:\n\nDid you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! Honey's unique composition, with an acidic pH and low water content, makes it a natural antibacterial agent that prevents the growth of bacteria and other microorganisms. This is why honey remains an incredibly versatile product that can be enjoyed for centuries without losing its flavor or nutritional value. Isn't that sweet?"

In [23]:
## Deepseek 
!ollama pull deepseek-r1:1.5b



pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.3 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.1 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.1 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 6.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 6.9 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  11 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  15 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  17 MB/1.1 GB              

In [24]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Sure! Here's a fun fact for you: \n\nThere are more ants in the world than humans. In fact, scientists estimate that there are between 5 and 12 billion ants on Earth. That’s over 5 million to 0.012 billion times more ants than humans!"